# Selenium 

In [2]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("chromedriver", chrome_options=options)

C:\Users\Max\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [65]:
import time
import requests
from bs4 import BeautifulSoup

URL = "https://tasty.co/topic/"
recipe_categories = ["breakfast", "lunch", "dinner", "desserts", "snacks"]

recipes_urls = []
n_recipes_per_cat = {cat : 0 for cat in recipe_categories}


for category in recipe_categories:
    print(f"Scraping {category}")
    full_url = URL + category 
    driver.get(full_url)
    show_more = driver.find_elements_by_css_selector("button.button--tasty.bold.xs-block.xs-mx-auto.xs-col-12.md-width-auto")
    while len(show_more) > 0:
        driver.execute_script("arguments[0].click();", show_more[0])
        time.sleep(3)
        show_more = driver.find_elements_by_css_selector("button.button--tasty.bold.xs-block.xs-mx-auto.xs-col-12.md-width-auto")
            
    page_source = driver.page_source
    
    soup = BeautifulSoup(page_source, "lxml")
    results = soup.findAll("a")
    for link in results:
        if "/recipe" in link["href"]:
            recipes_urls.append(link["href"])
            n_recipes_per_cat[category] += 1

Scraping breakfast
Scraping lunch
Scraping dinner
Scraping desserts
Scraping snacks


In [66]:
n_recipes_per_cat

{'breakfast': 627, 'lunch': 783, 'dinner': 760, 'desserts': 260, 'snacks': 140}

In [67]:
len(recipes_urls)

2570

In [68]:
import pickle
with open('url_list.pkl', 'wb') as handle:
    pickle.dump(recipes_urls, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('url_list.pkl', 'rb') as handle:
#     recipes_urls = pickle.load(handle)

In [70]:
len(recipes_urls)

2570

# 45 min

In [75]:
import requests
from bs4 import BeautifulSoup


def get_ingredients_list(url):
    ingredients = []
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all("ul", class_ = "list-unstyled xs-text-3")
    if len(results) > 0:
        results = results[0].find_all("li", class_ = "ingredient xs-mb1 xs-mt0")
        
        for ingedient in results:
            ingredients.append(ingedient.text)
    
    return ingredients

recipes_ingredients = []

for url in recipes_urls:
    full_url = "https://tasty.co/" + url
    recipes_ingredients.append(get_ingredients_list(full_url))

In [76]:
sum([len(ingrs) for ingrs in recipes_ingredients])

23153

In [77]:
import pickle
with open('tasty_ingredients.pkl', 'wb') as handle:
    pickle.dump(recipes_ingredients, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('tasty_ingredients.pkl', 'rb') as handle:
#     recipes_ingredients = pickle.load(handle)